**Train**

In [1]:
!pip install pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 6.4 MB/s eta 0:00:00a 0:00:01


In [5]:
import os
import numpy as np
import torch
from transformers import BlipProcessor, BlipModel
from PIL import Image
from pycocotools.coco import COCO

no_label_count = 0

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# File paths and directories (Kaggle specific)
IMAGE_DIR = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
ANNOTATION_FILE = '/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_train2017.json'
OUTPUT_PATH = '/kaggle/working'

# Load COCO annotations
coco = COCO(ANNOTATION_FILE)
image_ids = coco.getImgIds()

# Load the pretrained BLIP Processor and BLIP model
processor = BlipProcessor.from_pretrained("Salesforce/blip-itm-large-coco")
model = BlipModel.from_pretrained("Salesforce/blip-itm-large-coco")

# Move model to GPU if available
model.to(device)

def extract_features(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt")
        
        # Move inputs to GPU if available
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.get_image_features(**inputs)
            features = outputs.squeeze().cpu().numpy()
        
        return features
    
    except Exception as e:
        print(f"Error extracting features for image {image_path}: {str(e)}")
        return np.array([])  # Return empty array on error

# Feature extraction loop
features_list = []
labels_list = []
images_without_labels = []

# Check if there are any previously saved features and labels
if os.path.exists(os.path.join(OUTPUT_PATH, "checkpoint.npy")):
    checkpoint = np.load(os.path.join(OUTPUT_PATH, "checkpoint.npy"), allow_pickle=True).item()
    start_index = checkpoint['last_processed_index']
    features_list = checkpoint['features_list']
    labels_list = checkpoint['labels_list']
    no_label_count = checkpoint['no_label_count']
    images_without_labels = checkpoint['images_without_labels']
    print(f"Checkpoint found. Resuming from index {start_index}.")

else:
    start_index = 0

for i, img_id in enumerate(image_ids[start_index:], start=start_index):
    img_info = coco.loadImgs(img_id)[0]
    image_path = os.path.join(IMAGE_DIR, img_info['file_name'])
    features = extract_features(image_path)
    
    if features.size == 0:
        print(f"Empty features for image: {image_path}")
        continue
    
    # Get annotations for the image
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    label = [ann['category_id'] for ann in anns]  
    
    if not label:
        no_label_count += 1
        images_without_labels.append(image_path)
        continue

    features_list.append(features)
    labels_list.append(label)
    
    if (i + 1) % 11800 == 0:
        print(f"{i + 1} images processed")

    # Save checkpoint every 500 images processed
    if (i + 1) % 500 == 0:
        checkpoint = {
            'last_processed_index': i + 1,
            'features_list': features_list,
            'labels_list': labels_list,
            'no_label_count': no_label_count,
            'images_without_labels': images_without_labels
        }
        np.save(os.path.join(OUTPUT_PATH, "checkpoint.npy"), checkpoint)

# Flatten the features_list if necessary
train_features_array = np.vstack(features_list)

# Convert labels_list to one-hot encoding
unique_labels = sorted(set(label for labels in labels_list for label in labels))
label_map = {label: idx for idx, label in enumerate(unique_labels)}

def convert_to_one_hot(labels, label_map):
    one_hot_labels = np.zeros(len(label_map))
    for label in labels:
        if label in label_map:  # Ensure label is in the label map
            one_hot_labels[label_map[label]] = 1
    return one_hot_labels

one_hot_labels_list = [convert_to_one_hot(labels, label_map) for labels in labels_list]
train_labels_array = np.array(one_hot_labels_list)

# Ensure train_features_array and train_labels_array are correctly shaped
print("Features array shape:", train_features_array.shape)
print("Labels array shape:", train_labels_array.shape)

# Save final features and labels
os.makedirs(OUTPUT_PATH, exist_ok=True)  # Ensure OUTPUT_PATH exists
np.save(os.path.join(OUTPUT_PATH, "blip_train_features.npy"), train_features_array)
np.save(os.path.join(OUTPUT_PATH, "blip_train_labels.npy"), train_labels_array)

# Print images without labels
print("Total Images without labels: ", no_label_count)
print("Images without labels: ", images_without_labels)

# Remove checkpoint file if processing completed successfully
if os.path.exists(os.path.join(OUTPUT_PATH, "checkpoint.npy")):
    os.remove(os.path.join(OUTPUT_PATH, "checkpoint.npy"))
    print("Removed checkpoint file.")


Using device: cuda
loading annotations into memory...
Done (t=24.72s)
creating index...
index created!


Some weights of BlipModel were not initialized from the model checkpoint at Salesforce/blip-itm-large-coco and are newly initialized: ['logit_scale', 'text_model.embeddings.LayerNorm.bias', 'text_model.embeddings.LayerNorm.weight', 'text_model.embeddings.position_embeddings.weight', 'text_model.embeddings.word_embeddings.weight', 'text_model.encoder.layer.0.attention.output.LayerNorm.bias', 'text_model.encoder.layer.0.attention.output.LayerNorm.weight', 'text_model.encoder.layer.0.attention.output.dense.bias', 'text_model.encoder.layer.0.attention.output.dense.weight', 'text_model.encoder.layer.0.attention.self.key.bias', 'text_model.encoder.layer.0.attention.self.key.weight', 'text_model.encoder.layer.0.attention.self.query.bias', 'text_model.encoder.layer.0.attention.self.query.weight', 'text_model.encoder.layer.0.attention.self.value.bias', 'text_model.encoder.layer.0.attention.self.value.weight', 'text_model.encoder.layer.0.crossattention.output.LayerNorm.bias', 'text_model.encoder

EOFError: No data left in file

**Test**

In [11]:
import os
import numpy as np
import torch
from transformers import BlipProcessor, BlipModel
from PIL import Image
from pycocotools.coco import COCO

no_label_count: int = 0

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# File paths and directories (Kaggle specific)
IMAGE_DIR = '/kaggle/input/coco-2017-dataset/coco2017/test2017'
ANNOTATION_FILE = '/kaggle/input/annotation-2017/image_info_test2017.json'
OUTPUT_PATH = '/kaggle/working'

# Load COCO annotations
coco = COCO(ANNOTATION_FILE)
image_ids = coco.getImgIds()

# Load the pretrained BLIP Processor and BLIP model
processor = BlipProcessor.from_pretrained("Salesforce/blip-itm-large-coco")
model = BlipModel.from_pretrained("Salesforce/blip-itm-large-coco")

# Move model to GPU if available
model.to(device)

def extract_features(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    
    # Move inputs to GPU if available
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
        features = outputs.squeeze().cpu().numpy()
    
    return features

# Feature extraction loop
features_list = []
labels_list = []
images_without_labels = []

for i, img_id in enumerate(image_ids):
    img_info = coco.loadImgs(img_id)[0]
    image_path = os.path.join(IMAGE_DIR, img_info['file_name'])
    features = extract_features(image_path)
    
    # Ensure features are not empty
    if features.size == 0:
        print(f"Empty features for image: {image_path}")
        continue
    
    # Get annotations for the image
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    label = [ann['category_id'] for ann in anns]  
    
    if not label:
        no_label_count += 1
        images_without_labels.append(image_path)
        continue

    features_list.append(features)
    labels_list.append(label)
    
    if i % 4000 == 0:
        print(f"{i} images processed")

# Flatten the features_list if necessary
test_features_array = np.vstack(features_list)

# Convert labels_list to one-hot encoding
unique_labels = sorted(set(label for labels in labels_list for label in labels))
label_map = {label: idx for idx, label in enumerate(unique_labels)}

def convert_to_one_hot(labels, label_map):
    one_hot_labels = np.zeros(len(label_map))
    for label in labels:
        if label in label_map:  # Ensure label is in the label map
            one_hot_labels[label_map[label]] = 1
    return one_hot_labels

one_hot_labels_list = [convert_to_one_hot(labels, label_map) for labels in labels_list]
test_labels_array = np.array(one_hot_labels_list)

# Ensure train_features_array and train_labels_array are correctly shaped
print("Features array shape:", test_features_array.shape)
print("Labels array shape:", test_labels_array.shape)

# Save features and labels
np.save(os.path.join(OUTPUT_PATH, "blip_test_features.npy"), test_features_array)
np.save(os.path.join(OUTPUT_PATH, "blip_test_labels.npy"), test_labels_array)

# Print images without labels
print("Total Images without labels: ", no_label_count)
print("Images without labels: ", images_without_labels)


ImportError: cannot import name 'BlipForImageCaptioning' from 'transformers' (/opt/conda/lib/python3.10/site-packages/transformers/__init__.py)

**Val**

In [17]:
import os
import numpy as np
import torch
from transformers import BlipProcessor, BlipModel
from PIL import Image
from pycocotools.coco import COCO

no_label_count = 0

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# File paths and directories (Kaggle specific)
IMAGE_DIR = '/kaggle/input/coco-2017-dataset/coco2017/val2017'
ANNOTATION_FILE = '/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_val2017.json'
OUTPUT_PATH = '/kaggle/working'

# Load COCO annotations
coco = COCO(ANNOTATION_FILE)
image_ids = coco.getImgIds()

# Load the pretrained BLIP Processor and BLIP model
processor = BlipProcessor.from_pretrained("Salesforce/blip-itm-large-coco")
model = BlipModel.from_pretrained("Salesforce/blip-itm-large-coco")

# Move model to GPU if available
model.to(device)

def extract_features(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt")
        
        # Move inputs to GPU if available
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.get_image_features(**inputs)
            features = outputs.squeeze().cpu().numpy()
        
        return features
    
    except Exception as e:
        print(f"Error extracting features for image {image_path}: {str(e)}")
        return np.array([])  # Return empty array on error

# Feature extraction loop
features_list = []
labels_list = []
images_without_labels = []

# Check if there are any previously saved features and labels
if os.path.exists(os.path.join(OUTPUT_PATH, "checkpoint.npy")):
    checkpoint = np.load(os.path.join(OUTPUT_PATH, "checkpoint.npy"), allow_pickle=True).item()
    start_index = checkpoint['last_processed_index']
    features_list = checkpoint['features_list']
    labels_list = checkpoint['labels_list']
    no_label_count = checkpoint['no_label_count']
    images_without_labels = checkpoint['images_without_labels']
    print(f"Checkpoint found. Resuming from index {start_index}.")

else:
    start_index = 0

for i, img_id in enumerate(image_ids[start_index:], start=start_index):
    img_info = coco.loadImgs(img_id)[0]
    image_path = os.path.join(IMAGE_DIR, img_info['file_name'])
    features = extract_features(image_path)
    
    if features.size == 0:
        print(f"Empty features for image: {image_path}")
        continue
    
    # Get annotations for the image
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    label = [ann['category_id'] for ann in anns]  
    
    if not label:
        no_label_count += 1
        images_without_labels.append(image_path)
        continue

    features_list.append(features)
    labels_list.append(label)
    
    if (i + 1) % 500 == 0:
        print(f"{i + 1} images processed")

    # Save checkpoint every 10000 images processed
    if (i + 1) % 500 == 0:
        checkpoint = {
            'last_processed_index': i + 1,
            'features_list': features_list,
            'labels_list': labels_list,
            'no_label_count': no_label_count,
            'images_without_labels': images_without_labels
        }
        np.save(os.path.join(OUTPUT_PATH, "checkpoint.npy"), checkpoint)

# Flatten the features_list if necessary
val_features_array = np.vstack(features_list)

# Convert labels_list to one-hot encoding
unique_labels = sorted(set(label for labels in labels_list for label in labels))
label_map = {label: idx for idx, label in enumerate(unique_labels)}

def convert_to_one_hot(labels, label_map):
    one_hot_labels = np.zeros(len(label_map))
    for label in labels:
        if label in label_map:  # Ensure label is in the label map
            one_hot_labels[label_map[label]] = 1
    return one_hot_labels

one_hot_labels_list = [convert_to_one_hot(labels, label_map) for labels in labels_list]
val_labels_array = np.array(one_hot_labels_list)

# Ensure train_features_array and train_labels_array are correctly shaped
print("Features array shape:", val_features_array.shape)
print("Labels array shape:", val_labels_array.shape)

# Save final features and labels
os.makedirs(OUTPUT_PATH, exist_ok=True)  # Ensure OUTPUT_PATH exists
np.save(os.path.join(OUTPUT_PATH, "blip_val_features.npy"), val_features_array)
np.save(os.path.join(OUTPUT_PATH, "blip_val_labels.npy"), val_labels_array)

# Print images without labels
print("Total Images without labels: ", no_label_count)
print("Images without labels: ", images_without_labels)

# Remove checkpoint file if processing completed successfully
if os.path.exists(os.path.join(OUTPUT_PATH, "checkpoint.npy")):
    os.remove(os.path.join(OUTPUT_PATH, "checkpoint.npy"))
    print("Removed checkpoint file.")


Using device: cuda
loading annotations into memory...
Done (t=0.78s)
creating index...
index created!


Some weights of BlipModel were not initialized from the model checkpoint at Salesforce/blip-itm-large-coco and are newly initialized: ['logit_scale', 'text_model.embeddings.LayerNorm.bias', 'text_model.embeddings.LayerNorm.weight', 'text_model.embeddings.position_embeddings.weight', 'text_model.embeddings.word_embeddings.weight', 'text_model.encoder.layer.0.attention.output.LayerNorm.bias', 'text_model.encoder.layer.0.attention.output.LayerNorm.weight', 'text_model.encoder.layer.0.attention.output.dense.bias', 'text_model.encoder.layer.0.attention.output.dense.weight', 'text_model.encoder.layer.0.attention.self.key.bias', 'text_model.encoder.layer.0.attention.self.key.weight', 'text_model.encoder.layer.0.attention.self.query.bias', 'text_model.encoder.layer.0.attention.self.query.weight', 'text_model.encoder.layer.0.attention.self.value.bias', 'text_model.encoder.layer.0.attention.self.value.weight', 'text_model.encoder.layer.0.crossattention.output.LayerNorm.bias', 'text_model.encoder

500 images processed
1000 images processed
1500 images processed
2000 images processed
2500 images processed
3000 images processed
3500 images processed
4000 images processed
4500 images processed
5000 images processed
Features array shape: (4952, 512)
Labels array shape: (4952, 80)
Total Images without labels:  48
Images without labels:  ['/kaggle/input/coco-2017-dataset/coco2017/val2017/000000226111.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000058636.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000458790.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000461275.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000268996.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000370999.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000481404.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000121153.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000310622.jpg', '/kaggle/input/coco-2017-dataset/coco

**FCN with 1DCNN**

In [ ]:
import os
import pandas as pd
import numpy as np
import torchaudio
import torch
import tensorflow as tf
import sklearn.metrics


train_features = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/train/wavlm_train_features.npy")
train_labels = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/train/wavlm_train_labels.npy")
val_features = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/val/wavlm_val_features.npy")
val_labels = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/val/wavlm_val_labels.npy")
test_features = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/test/wavlm_test_features.npy")
test_labels = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/test/wavlm_test_labels.npy")
print(train_features.shape)
print(train_labels.shape)
print(val_features.shape)
print(val_labels.shape)
print(test_features.shape)
print(test_labels.shape)
train_features = np.expand_dims(train_features, axis=2)
val_features = np.expand_dims(val_features, axis=2)
test_features = np.expand_dims(test_features, axis=2)
print(train_features.shape)
print(val_features.shape)
print(test_features.shape)

In [ ]:
def create_fcn_model(input_shape, hidden_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hidden_size, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(hidden_size // 2, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(hidden_size // 4, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(hidden_size // 8, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
    ])
    return model

# Define model hyperparameters
input_shape = (768,1) #WavLM
hidden_size = 1024
learning_rate = 0.0005
num_epochs = 20
batch_size = 32
model = create_fcn_model(input_shape, hidden_size)
print(model.summary())

In [ ]:
# Define learning rate scheduling callback
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)


# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Train the FCN model with learning rate scheduling
model.fit(train_features, train_labels,
          epochs=num_epochs,
          batch_size=batch_size,
          verbose=1,
          validation_data=(val_features, val_labels),
          callbacks=[lr_scheduler])

In [ ]:
predicted_scores = model.predict(test_features)  
# Create a new NumPy array with the condition
#pred_labels = tf.math.argmax(predicted_scores, axis=1)
print(predicted_scores.shape)


# Define the compute_eer function
def compute_eer(label, pred, positive_label=1):
    fpr, tpr, threshold = sklearn.metrics.roc_curve(label, pred, pos_label=positive_label)
    fnr = 1 - tpr
    eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer = (eer_1 + eer_2) / 2
    return eer

# Calculate EER using the compute_eer function
eer = compute_eer(test_labels, predicted_scores)

print("Equal Error Rate (EER):", eer)

**FCN Without 1D CNN**

In [ ]:
import os
import pandas as pd
import numpy as np
import torchaudio
import torch
import tensorflow as tf
import sklearn.metrics

train_features = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/train/wavlm_train_features.npy")
train_labels = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/train/wavlm_train_labels.npy")
val_features = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/val/wavlm_val_features.npy")
val_labels = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/val/wavlm_val_labels.npy")
test_features = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/test/wavlm_test_features.npy")
test_labels = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/test/wavlm_test_labels.npy")
print(train_features.shape)
print(train_labels.shape)
print(val_features.shape)
print(val_labels.shape)
print(test_features.shape)
print(test_labels.shape)
train_features = np.expand_dims(train_features, axis=2)
val_features = np.expand_dims(val_features, axis=2)
test_features = np.expand_dims(test_features, axis=2)
print(train_features.shape)
print(val_features.shape)
print(test_features.shape)

In [ ]:
# Define the FCN model architecture without 1DCNN
def create_fcn_model(input_shape, hidden_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=input_shape),  # Flatten the input
        tf.keras.layers.Dense(hidden_size, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(hidden_size // 2, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(hidden_size // 4, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(hidden_size // 8, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Define model hyperparameters
input_shape = (768,1) #WavLM
hidden_size = 1024
learning_rate = 0.0005
num_epochs = 20
batch_size = 32
model = create_fcn_model(input_shape, hidden_size)
print(model.summary())

In [ ]:
# Define learning rate scheduling callback
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)


# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Train the FCN model with learning rate scheduling
model.fit(train_features, train_labels,
          epochs=num_epochs,
          batch_size=batch_size,
          verbose=1,
          validation_data=(val_features, val_labels),
          callbacks=[lr_scheduler])

In [ ]:
predicted_scores = model.predict(test_features)  
# Create a new NumPy array with the condition
#pred_labels = tf.math.argmax(predicted_scores, axis=1)
print(predicted_scores.shape)


# Define the compute_eer function
def compute_eer(label, pred, positive_label=1):
    fpr, tpr, threshold = sklearn.metrics.roc_curve(label, pred, pos_label=positive_label)
    fnr = 1 - tpr
    eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer = (eer_1 + eer_2) / 2
    return eer

# Calculate EER using the compute_eer function
eer = compute_eer(test_labels, predicted_scores)

print("Equal Error Rate (EER):", eer)